# Conexión a Base de Datos y Carga Inicial de Datos

Este notebook forma parte del flujo de trabajo para el análisis de datos. Su objetivo es conectar con la base de datos y extraer los datos brutos para su posterior procesamiento.

Para empezar se importan las librerias necesarias para el funcionamiento correcto del código

In [1]:
import psycopg2
import json
import os

## Configuración de la Conexión
A continuación, se leen y extraen las credenciales para hacer la conexión a PostgreSQL. Seguido de la realización de la conexión a la base de datos almacenada en PostgreSQL. 

In [ ]:
with open("credentials.json", "r", encoding="utf-8") as file:
    credentials = json.load(file)

In [ ]:
db_host = credentials["db_host"]
db_name = credentials["db_name"]
db_user = credentials["db_user"]
db_password = credentials["db_password"]
db_port = credentials["db_port"] 

conn = psycopg2.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)

Se establece un cursor que permite ejecutar comandos SQL, para seguidamente hacer la creación de la tabla en la base de datos que contenga los datos sin procesar. 

In [ ]:
cur = conn.cursor()

cur.execute("""CREATE TABLE leukemia_raw_data (
            patient_id SERIAL PRIMARY KEY,
            age INTEGER,
            gender TEXT ,
            country TEXT,
            wbc_count INTEGER,
            rbc_count NUMERIC(5,2),
            platelet_count INTEGER,
            hemoglobin_level NUMERIC(5,2),
            bone_marrow_blasts INTEGER,
            genetic_mutation TEXT,
            family_history TEXT,
            smoking_status TEXT,
            alcohol_consumption TEXT,
            radiation_exposure TEXT,
            infection_history TEXT,
            bmi NUMERIC(5,2),
            chronic_illness TEXT,
            immune_disorders TEXT,
            ethnicity TEXT,
            socioeconomic_status TEXT,
            urban_rural TEXT,
            leukemia_status TEXT
        );
""")
conn.commit()

In [5]:
os.chdir("..")
print(os.getcwd())

c:\Users\valen\Desktop\Leukemia-Cancer-Risk-ETL


Finalmente, se construye la ruta del archivo CSV y se hace la carga de datos a la tabla creada previamente.

In [6]:
csv_file_path = os.path.join(os.getcwd(), "data", "biased_leukemia_dataset.csv")

table_name = 'leukemia_raw_data'

copy_sql = f"""
           COPY {table_name} FROM stdin 
           DELIMITER as ','
           CSV HEADER
           """
with open(csv_file_path, 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)

conn.commit()

conn.close()